In [45]:
import os
import glob
from pyspark.sql.window import Window
from pyspark.sql import SparkSession
from pyspark.sql import functions
from  pyspark.sql.types import StructType, StructField, StringType, IntegerType
from datetime import datetime
import time 
from pyspark.sql.functions import col, split, when, size,count,row_number,desc,trim,regexp_replace

# Define the window specification

os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'notebook'
print("completed")

file_pattern = r"D:/DataInc/PDL-1-2B-Splitted/1.2 Billion PeopleDataLabs Database-07*.csv"
file_pattern1 =r"D:/DataInc/PDL-1-2B-Splitted/1.2 Billion PeopleDataLabs Database-08*.csv"
file_pattern2 =r"D:/DataInc/PDL-1-2B-Splitted/1.2 Billion PeopleDataLabs Database-09*.csv"
file_pattern3 =r"D:/DataInc/PDL-1-2B-Splitted/1.2 Billion PeopleDataLabs Database-1*.csv"
files = glob.glob(file_pattern)
files1 = glob.glob(file_pattern1)
files2 = glob.glob(file_pattern2)
files3 = glob.glob(file_pattern3)
#print(f"Number of matched files: {len(files)}")
file=files+files1+files2+files3
print(file)
print(f"Number of matched files: {len(file)}")

completed
['D:/DataInc/PDL-1-2B-Splitted\\1.2 Billion PeopleDataLabs Database-070.csv', 'D:/DataInc/PDL-1-2B-Splitted\\1.2 Billion PeopleDataLabs Database-071.csv', 'D:/DataInc/PDL-1-2B-Splitted\\1.2 Billion PeopleDataLabs Database-072.csv', 'D:/DataInc/PDL-1-2B-Splitted\\1.2 Billion PeopleDataLabs Database-073.csv', 'D:/DataInc/PDL-1-2B-Splitted\\1.2 Billion PeopleDataLabs Database-074.csv', 'D:/DataInc/PDL-1-2B-Splitted\\1.2 Billion PeopleDataLabs Database-075.csv', 'D:/DataInc/PDL-1-2B-Splitted\\1.2 Billion PeopleDataLabs Database-076.csv', 'D:/DataInc/PDL-1-2B-Splitted\\1.2 Billion PeopleDataLabs Database-077.csv', 'D:/DataInc/PDL-1-2B-Splitted\\1.2 Billion PeopleDataLabs Database-078.csv', 'D:/DataInc/PDL-1-2B-Splitted\\1.2 Billion PeopleDataLabs Database-079.csv', 'D:/DataInc/PDL-1-2B-Splitted\\1.2 Billion PeopleDataLabs Database-080.csv', 'D:/DataInc/PDL-1-2B-Splitted\\1.2 Billion PeopleDataLabs Database-081.csv', 'D:/DataInc/PDL-1-2B-Splitted\\1.2 Billion PeopleDataLabs Databas

In [46]:


try:
    # Initialize SparkSession for local mode
    spark = SparkSession.builder.appName("LocalPySparkTest").config("spark.driver.extraClassPath", "C:/Users/Administrator/Downloads/postgresql-42.7.4.jar")\
    .config("spark.driver.memory", "200g")\
    .config("spark.sql.shuffle.partitions", "480") \
    .master("local[*]")\
    .getOrCreate()
    #
    

    shema = StructType([
    StructField("Name", StringType(), True),  # Replace with your column names and data types
    StructField("Email_List", StringType(), True),
    StructField("Phone_List", StringType(), True),
    StructField("LinkedInURL", StringType(), True),
    StructField("Address", StringType(), True)
    ])
    # Create a DataFrame
    #rdd = spark.sparkContext.parallelize(data)
    #file_path = r"C:/Users/Administrator/Downloads/tasks.csv"
    #file_path=r"D:/DataInc/PDL-1-2B-Splitted/1.2 Billion PeopleDataLabs Database-000.csv"
    file_path=r"D:/DataInc/PDL-1-2B-Splitted/1.2 Billion PeopleDataLabs Database-12*.csv"

    # Read the CSV file into a DataFrame
    #df = spark.read.csv(file_path, header=True, inferSchema=True)
    print("Spark started reading the file")
    start_time = datetime.now()
    dfs=[spark.read.format("csv").option("header", "false").option("inferSchema", "false").option("delimiter", ",") \
    .option("quote", "\"").option("escape", "\\") \
    .option("lineSep", "\n")\
    .schema(shema)\
    .load(f) for f in file]
    # Print success message
    df = dfs[0]
    for other_df in dfs[1:]:
        df = df.union(other_df)
    
    # Record the end time
    end_time = datetime.now()
    print(f"Process ended at: {end_time}")
    
    # Calculate the duration
    duration = end_time - start_time
    print(f"Total duration: {duration}")
    print("PySpark job ran successfully on the local machine.")

except Exception as e:
    # Print error message if job fails
    print(f"PySpark job failed with error: {e}")

print(spark.sparkContext._conf.getAll())

Spark started reading the file
Process ended at: 2024-12-01 01:34:44.488909
Total duration: 0:00:01.248443
PySpark job ran successfully on the local machine.
[('spark.app.name', 'LocalPySparkTest'), ('spark.sql.shuffle.partitions', '480'), ('spark.app.startTime', '1733041680668'), ('spark.executor.id', 'driver'), ('spark.app.id', 'local-1733041682552'), ('spark.sql.warehouse.dir', 'file:/C:/spark/spark-3.1.2-bin-hadoop3.2/spark-warehouse'), ('spark.rdd.compress', 'True'), ('spark.driver.extraClassPath', 'C:/Users/Administrator/Downloads/postgresql-42.7.4.jar'), ('spark.serializer.objectStreamReset', '100'), ('spark.driver.port', '55505'), ('spark.driver.host', 'HV01'), ('spark.driver.memory', '200g'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.submit.deployMode', 'client'), ('spark.ui.showConsoleProgress', 'true')]


In [47]:



df=df.filter(((col('Email_List').isNotNull()) &  (col('Email_List') != "") & (col('Email_List') != "null")) | ((col('Phone_List').isNotNull()) &  (col('Phone_List') != "") &  (col('Phone_List') != "null")))
windowSpec = Window.partitionBy("LinkedInURL").orderBy(col("Email_List").asc())
df = df.withColumn("row_number", row_number().over(windowSpec)).filter(col('row_number')==1).drop("row_number").filter(col("LinkedInURL").isNotNull())
print("deleted duplicate and null records")
#df.show()

deleted duplicate and null records


In [48]:
#df.select(*df.columns[:4]).count()
#df.count()#select(['_c0','_c4']).show(4,truncate=False)

#columns=["name","email_list","phone_list","URL","address"]
#df.toDF(*columns)
#df.rdd.getNumPartitions()#show(4,truncate=False)
#df1=df.groupBy(col("URL")).agg(count('*').alias('count_d'))
#df1.filter(col('count_d')>1).show(truncate=False)
#df.count()
#print(spark.sparkContext.getConf().get("spark.driver.cores"))
#print(spark.sparkContext.getConf().get("spark.executor.cores"))
num_cores = spark.sparkContext.defaultParallelism

print(f"Number of cores available: {num_cores}")

Number of cores available: 24


In [49]:
#first_name last _name deriving Logic
#df.select("name").filter(size(split(col("name")," "))>4).
#split the name into fname and lname

#df_name=df.withColumn("Fname",split(col("name")," ").getItem(0)).withColumn("Lname",split(col("name")," ",2).getItem(1))
# Split the name column into Fname and Lname, and trim the results
df_name = df.withColumn("Fname", trim(split(col("Name"), " ").getItem(0))) \
            .withColumn("Lname", trim(split(col("Name"), " ", 2).getItem(1)))

# Show the result
#df_name.select("name", "Fname", "Lname").show(40,truncate=False)
#df.withColumn("split_email",split(col("Email_list"),",")).select(['Email_list','split_email']).show(200,truncate=False)
df_email_split = df_name.withColumn("email_array", split(col("Email_List"), ","))

# Derive WEmail (index 0) and PEmail (index 1)
df_emails = df_email_split \
    .withColumn("WEmail", when(size(col("email_array")) > 0, trim(col("email_array")[0])).otherwise(None)) \
    .withColumn("PEmail", when(size(col("email_array")) > 1, trim(col("email_array")[1])).otherwise(None)).drop("email_array")
#df_emails.select(["email_list","WEmail","PEmail"]).show(200,truncate=False)



df_split = df_emails.withColumn("phone_array", split(col("Phone_List"), ","))

# Derive WPhone, HPhone, and MPhone with trimming
df_phones = df_split \
    .withColumn("WPhone", when(size(col("phone_array")) > 0, trim(col("phone_array")[0])).otherwise(None)) \
    .withColumn("HPhone", when(size(col("phone_array")) > 1, trim(col("phone_array")[1])).otherwise(None)) \
    .withColumn("MPhone", when(size(col("phone_array")) > 2, trim(col("phone_array")[2])).otherwise(None)).drop("phone_array")


# Show the result
#df_phones.select("phone_list", "WPhone", "HPhone", "MPhone").show(200,truncate=False)
print("emails and phone numbers and names are transformed")

emails and phone numbers and names are transformed


In [50]:
df = df_phones.withColumn("split_location", split(col("Address"), ",")) 
#df.select(['address','split_location']).where(size(col("split_location"))>3).show(100,truncate=False)
df = df.withColumn("Country",when(size(col("split_location")) == 4, trim(col("split_location")[3]))
    .when(size(col("split_location")) == 3, trim(col("split_location")[2]))
    .when(size(col("split_location")) == 2, trim(col("split_location")[1]))
    .when(size(col("split_location")) == 1, trim(col("split_location")[0]))
    .otherwise(None)
)

df = df.withColumn("State",when(size(col("split_location")) == 4, trim(col("split_location")[1]))
    .when(size(col("split_location")) == 3, trim(col("split_location")[1]))
    .when(size(col("split_location")) == 2, trim(col("split_location")[1]))
    .otherwise(None)
)

df = df.withColumn(
    "City",when(size(col("split_location")) == 4 , trim(col("split_location")[0]))
    .when(size(col("split_location")) == 3 , trim(col("split_location")[0]))
    .otherwise(None)
)

df_final=df.drop("split_location")#.where(size(col("split_location"))>3).show(100,truncate=False)
print("address transformation completed")

address transformation completed


In [51]:
print("clean up start")
for column in df_final.columns:
    df_final = df_final.withColumn(column, regexp_replace(col(column), "\\x00", ""))
print("clean up done")

clean up start
clean up done


In [52]:
jdbc_url = "jdbc:postgresql://localhost:5432/postgres"  # Replace with your details
connection_properties = {
    "user": "postgres",         # Replace with your username
    "password": "*****", # Replace with your password
    "driver": "org.postgresql.Driver"
}
start_time = datetime.now()

print(f"Process started at: {start_time}")


df_final.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "pdl_data").options(**connection_properties) \
    .mode("append").save()  # Use 'overwrite', 'append', or 'ignore' based on your use case

# Record the end time
end_time = datetime.now()
print(f"Process ended at: {end_time}")

# Calculate the duration
duration = end_time - start_time
print(f"Total duration: {duration}")
print("data has been written to master table in postgress")

"""
# Load data from PostgreSQL table
df = spark.read \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "test").options(**connection_properties) \
    .load()
"""
# Show data
#df.show()

Process started at: 2024-12-01 01:34:45.362738
Process ended at: 2024-12-01 02:15:28.935766
Total duration: 0:40:43.573028
data has been written to master table in postgress


'\n# Load data from PostgreSQL table\ndf = spark.read     .format("jdbc")     .option("url", jdbc_url)     .option("dbtable", "test").options(**connection_properties)     .load()\n'

In [53]:
df_final.count()

178740352